# Outputs export


Several utilities are available for exporting model outputs in useful formats. 

We will go through the main ones.

In [ ]:
import os
from script import readOutput as rout

import matplotlib
import pyvista as pv
import matplotlib.pyplot as plt

label_size = 7
matplotlib.rcParams['xtick.labelsize'] = label_size
matplotlib.rcParams['ytick.labelsize'] = label_size
matplotlib.rc('font', size=6)

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

We first define a folder where exported files will be stored:

In [ ]:
out_path = 'export'
if not os.path.exists(out_path):
    os.makedirs(out_path)

## Netcdf outputs

We start with the `netcdf` outputs as it is the most common one used in our field.

### Building netcdf file

`Netcdf` exports are done by using the `readOutput.py` script presented in the previous notebook. Here we export all the time steps at once by looping through the number of outputs (5 in this case).

:::{admonition} Arguments for `readOutput.py`
:class: note, dropdown

The `readOutput.py` script main function requires several arguments:

+ `path`: the path to the input file
+ `filename`: the name of the input file
+ `step`: the step you wish to output (here set to 5 corresponding to the last output based on the input parameters: start time 0 year, end time 50 thousand years with an output every 10 thousand years)
+ `nbstep`: the number of time steps to plot (useful if one want to output a `netdcf` file containing all time steps (done in the following section).
+ `uplift_forcing`: set to False as we are not considering any tectonic forcing


:::

Then the `buildLonLatMesh` function is used to interpolate (using a `kd-tree` approach) the `gospl` variables on a regular mesh. It also provides a way to limit the created `netcdf` file by defining a `bounding box`:

In [ ]:
uplift_forcing = False

# Specifying the grid resolution in degrees
reso = 0.1

# Total number of outputs
nbstep = 5

# Bounding box
bb = [-134,20,-46,80]

# Looping through the output time steps
for k in range(nbstep+1):
    if k == 0:
        # Calling the initialisation function for our class 
        ncgrid = rout.readOutput(path='./', filename='input.yml', 
                                 step=k, nbstep=nbstep+1, 
                                 uplift=uplift_forcing)
    else:
        # Update the variables after the first time steps
        ncgrid.update(step=k, uplift=uplift_forcing)
    
    # Build the regular grid defining the bounding box
    ncgrid.buildLonLatMesh(res=reso, nghb=3, box=bb)

Exporting the `netcdf` file on the desired bounding box:

In [ ]:
ncout = os.path.join(out_path, "GoMresult.nc")
ncgrid.exportNetCDF(ncfile = ncout)

### Visualisation with `ipygany`

Visualise the result in Jupyter environment with `ipygany`. This is done using by opening the `netcdf` file first:

In [ ]:
import numpy as np
import xarray as xr

from ipygany import PolyMesh, Scene, IsoColor, WarpByScalar, ColorBar, colormaps
from ipywidgets import VBox, FloatSlider, FileUpload, Dropdown, jslink

ds = xr.open_dataset(ncout, decode_times=False)
ds

We then select a specific time step and variable using `xarray` functions:

In [ ]:
# Selecting last time step
ds_z = ds.isel(time=[-1])

# Dropping all variables expect the elevation
ds_z = ds_z.drop(['time','erodep', 'precipitation', 'drainageArea', 'basinID'])
ds_z = ds_z.squeeze("time")
ds_z

:::{note}

We now create `pyvista` structured mesh (our `netcdf` is structured!)

:::

In [ ]:
xx, yy, zz = np.meshgrid(np.radians(ds_z['longitude']), 
                         np.radians(ds_z['latitude']), 
                         [0])

# Transform to spherical coordinates
radius = 6371.0e6
x = radius * np.cos(yy) * np.cos(xx)
y = radius * np.cos(yy) * np.sin(xx)
z = radius * np.sin(yy)

grid = pv.StructuredGrid(x, y, z)

# Add data to mesh
for var in ds_z.data_vars:
    grid[var] = np.array(ds_z[var]).ravel(order='F')

We can then convert `pyvista` mesh to `ipygany` mesh


````{margin}
```{seealso}
This approach is based on the following [cerege example](https://cerege-cl.github.io/Notebooks-Gallery/notebooks/ipygany_3d_example.html).
```
````

In [ ]:
# Turn the PyVista mesh into a PolyMesh
mesh = PolyMesh.from_vtk(grid)

# Color the mesh
colored_mesh = IsoColor(mesh, min=ds_z.elevation.min(), max=ds_z.elevation.max())

# setup warping
warped_mesh = WarpByScalar(colored_mesh, input='elevation', factor=0)

In [ ]:
# Link a slider to the warp value
warp_slider = FloatSlider(min=0., max=10., value=5)

def on_slider_change(change):
    warped_mesh.factor = change['new'] * -10000

warp_slider.observe(on_slider_change, 'value')

# Create a colorbar widget
colorbar = ColorBar(colored_mesh)

# Colormap choice widget
colormap = Dropdown(
    options=colormaps,
    description='colormap:'
)

jslink((colored_mesh, 'colormap'), (colormap, 'index'))

VBox((colormap, warp_slider, Scene([warped_mesh])))

## Geotiff output


### `xarray` functionality

Let first use the `netcdf` file created and open it with the `xarray` library in the jupyter environment:

In [ ]:
import xarray as xr

ds = xr.open_dataset(ncout, decode_times=False)
ds

By default the mesh is written in lon/lat (projection [epsg:4326](https://spatialreference.org/ref/epsg/wgs-84/) as `gospl` is a global model). 

Using the `rioxarray` library we have the ability to reproject the dataset in any other type of projection. Let's reproject the dataset in `utm` coordinates:

In [ ]:
import rioxarray

ds = ds.rio.write_crs(4326)
print('Default projection:',ds.rio.crs)

print('Estimated UTM projection:',ds.rio.estimate_utm_crs())
ds_utm = ds.rio.reproject(ds.rio.estimate_utm_crs())

ds_utm

Let's now create a `geotiff` file containing the elevation for the last time step using the `rioxarray` functionality:

In [ ]:
elevArray = ds_utm.isel(time=[-1])["elevation"][0,:,:]

# Export to geotiff
tifout = os.path.join(out_path, "GoMresult5.tif")

elevArray.rio.to_raster(tifout)

### Advanced functionalities on geotiff

We can use the `rasterio` library to visualise the `geotiff` file in our notebook:

In [ ]:
import rasterio
from rasterio.mask import mask
from rasterio.plot import show, show_hist

data = rasterio.open(tifout)

print('Number of band',data.count)
print('Image resolution',data.height, data.width)
print('CRS',data.crs)

In [ ]:
fig, (axr) = plt.subplots(1,1, figsize=(8,9))
show((data, 1), ax=axr, cmap='gist_earth', vmin=-10000, vmax=10000)
plt.show()

What we want to do next is to create a bounding box around the Gulf of Mexico region and clip the raster based on that.

We create a bounding box with `Shapely`.

In [ ]:
from shapely.geometry import box

# WGS84 coordinates (lon/lat - x/y)
minx, miny = -102.5, 18.5
maxx, maxy = -82.0, 40.0 
bbox = box(minx, miny, maxx, maxy)

We insert the bbox into a `GeoDataFrame` and re-project into the same coordinate system as the raster data

In [ ]:
import geopandas as gpd
from fiona.crs import from_epsg

geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(4326))
geo = geo.to_crs(crs=data.crs.data)

Next we need to get the coordinates of the geometry in such a format that rasterio wants them. This can be conducted easily with following function

In [ ]:
def getFeatures(gdf):
    
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

Get the geometry coordinates by using the function.

In [ ]:
coords = getFeatures(geo)
print(coords)

Now we are ready to clip the raster with the polygon using the coords variable that we just created. 

:::{tip}
Clipping the raster can be done easily with the `mask` function that we imported in the beginning from `rasterio`, and specifying `clip=True`.
:::

In [ ]:
out_img, out_transform = mask(data, shapes=coords, crop=True)

Next, we need to modify the metadata. 

Let’s start by copying the metadata from the original data file.

Then we parse the `EPSG` value from the `CRS` so that we can create a `Proj4` string using `PyCRS` library (to ensure that the projection information is saved correctly).

In [ ]:
import pycrs

out_meta = data.meta.copy()
print(out_meta)

epsg_code = int(data.crs.data['init'][5:])
print(epsg_code)


out_meta.update({"driver": "GTiff", "height": out_img.shape[1],
                 "width": out_img.shape[2], "transform": out_transform,
                 "crs": pycrs.parse.from_epsg_code(epsg_code).to_proj4()})

Finally, we can save the clipped raster to disk with following command.

In [ ]:
tifout2 = os.path.join(out_path, "GoM_clipped.tif")

with rasterio.open(tifout2, "w", **out_meta) as dest:
    dest.write(out_img)

Let’s still check that the result is correct by plotting our new clipped raster.

In [ ]:
clipped = rasterio.open(tifout2)

fig, (axr) = plt.subplots(1,1, figsize=(5,5))
show((clipped, 1), ax=axr, cmap='gist_earth', vmin=-1000, vmax=1000)
plt.show()

## ZMAP files

The `zmapio` library allows to read and write map gridded data using `ZMAP Plus ASCII Grid format`. 

Here we will use it to write our elevation as a ZMAP grid.

In [ ]:
from zmapio import ZMAPGrid

z_val = elevArray.values[::5,::5].T
print('Z-values shape: ', z_val.shape)

Define the ZMAP dataset:

In [ ]:
new_zgrid = ZMAPGrid(z_values=z_val, min_x=ds_utm.x.values.min(), 
                     max_x=ds_utm.x.values.max(),
                     min_y=ds_utm.y.values.min(),  
                     max_y=ds_utm.y.values.max())

Write the new ZMAP file by customising the formating:

In [ ]:
zgridout = os.path.join(out_path, "GoM_zmap.dat")

new_zgrid.comments = ['Model', 'output']
new_zgrid.nodes_per_line = 4
new_zgrid.field_width = 15
new_zgrid.decimal_places = 3
new_zgrid.name = 'gospl'
new_zgrid.write(zgridout)

In [ ]:
!head $zgridout

Let's visualise the dataset... 

In [ ]:
new_zgrid.plot(cmap='gist_earth', shading='auto')